# Figures Spatial examples

In [ ]:
# Libraries
import os
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cartopy.crs as ccrs
from matplotlib.ticker import FormatStrFormatter
from matplotlib.ticker import MaxNLocator

In [ ]:
# Directories
dir04 = '../paper_deficit/output/04_out/'
dir05 = '../paper_deficit/output/05_prep_other/'
dir06 = '../paper_deficit/output/06_eval/'
dir_nearth = '../data/naturalearth/'

---

In [ ]:
# Libraries
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
import dask

# Initialize dask
cluster = SLURMCluster(
    queue='compute',                      # SLURM queue to use
    cores=12,                             # Number of CPU cores per job
    memory='256 GB',                      # Memory per job
    account='bm0891',                     # Account allocation
    interface="ib0",                      # Network interface for communication
    walltime='01:00:00',                  # Maximum runtime per job
    local_directory='../dask/',           # Directory for local storage
    job_extra_directives=[                # Additional SLURM directives for logging
        '-o ../dask/LOG_worker_%j.o',     # Output log
        '-e ../dask/LOG_worker_%j.e'      # Error log
    ]
)

# Scale dask cluster
cluster.scale(jobs=2)

# Configurate dashboard url
dask.config.config.get('distributed').get('dashboard').update(
    {'link': '{JUPYTERHUB_SERVICE_PREFIX}/proxy/{port}/status'}
)

# Create client
client = Client(cluster)

client

In [ ]:
# Get coastline shapefile
coastline110 = gpd.read_file(
    dir_nearth + 'ne_110m_coastline/ne_110m_coastline.shp')

In [ ]:
# Get carbon densities from analysis and create cveg and csoil arrays
ds_agbc = xr.open_dataset(os.path.join(dir04, 'agbc.nc')) \
    .chunk(lat=5000, lon=5000).persist()
ds_bgbc = xr.open_dataset(os.path.join(dir04, 'bgbc.nc')) \
    .chunk(lat=5000, lon=5000).persist()
ds_soc = xr.open_dataset(os.path.join(dir04, 'soc.nc')) \
    .chunk(lat=5000, lon=5000).persist()

da_cveg_act = ds_agbc.agbc_max_act + ds_bgbc.bgbc_max_act
da_cveg_pot = ds_agbc.agbc_max_prim + ds_bgbc.bgbc_max_prim

da_soc_act = ds_soc.soc_mean_act
da_soc_pot = ds_soc.soc_mean_prim

In [ ]:
# Get prepared data from other studies
ds_erb = xr.open_zarr(
    os.path.join(dir05, 'fig_example/ds_erb_preped.zarr')).persist()
ds_sand = xr.open_zarr(
    os.path.join(dir05, 'fig_example/ds_sand_preped.zarr')).persist()
ds_walker = xr.open_zarr(
    os.path.join(dir05, 'fig_example/ds_walker_preped.zarr')).persist()
ds_mo = xr.open_zarr(
    os.path.join(dir05, 'fig_example/ds_mo_preped.zarr')).persist()

---

In [ ]:
# Plot functions
def plot_sub(fig, ax, da, vmax, lat_min, lat_max, lon_min, lon_max, cmap, 
             cbar=False, ax_cbar = False):
    """Prepare subplot"""
    # slice data
    da_sub = da.sel(lat=slice(lat_max, lat_min), 
                    lon=slice(lon_min, lon_max))

    # plot
    im = da_sub.plot.imshow(ax=ax, vmin=0, vmax=vmax, 
                             cmap=cmap, add_colorbar=False)
    # colorbar
    if cbar == True:
        cbar0 = fig.colorbar(im, ax=ax_cbar, orientation='vertical', 
                             extend='max', label='tC ha$^{-1}$', shrink=1, 
                             aspect=25, pad=0.02)   
        cbar0.ax.tick_params(size=0)
    return im


def plot_example(lat_min, lat_max, lon_min, lon_max, vmax_veg, vmax_soil, 
                 name_out=None):
    
    """Plot extracts of our data, Walker, Erb, and Sanderman"""
    
    # Coordinate selection
    coords = lat_min, lat_max, lon_min, lon_max

    # Plot
    fig = plt.figure(figsize=(9, 9), dpi=600, constrained_layout=True)
    fig.set_facecolor('#ffffff')

    gs = fig.add_gridspec(5, 4, 
                          hspace=0.1, wspace=0.1,
                          width_ratios=(0.25, 0.25, 0.25, 0.25), 
                          height_ratios=(0.2, 0.2, 0.2, 0.2, 0.2))

    axes = [fig.add_subplot(gs[i, j], aspect='auto') for i in range(5) for j in range(4)]

    # Subplots
    # Define all plot configurations
    plot_configs = [
        {'ax': axes[0], 'data': da_cveg_act, 'vmax': vmax_veg, 'cbar': True, 'cbar_ax': [axes[0], axes[1]]},
        {'ax': axes[1], 'data': da_cveg_pot, 'vmax': vmax_veg, 'cbar': False},
        {'ax': axes[2], 'data': da_soc_act, 'vmax': vmax_soil, 'cbar': True, 'cbar_ax': [axes[2], axes[3]]},
        {'ax': axes[3], 'data': da_soc_pot, 'vmax': vmax_soil, 'cbar': False},
        {'ax': axes[4], 'data': ds_mo.mo_cveg_act, 'vmax': vmax_veg, 'cbar': True, 'cbar_ax': [axes[4], axes[5]]},
        {'ax': axes[5], 'data': ds_mo.mo_cveg_pot, 'vmax': vmax_veg, 'cbar': False},
        {'ax': axes[8], 'data': ds_walker.cveg_act, 'vmax': vmax_veg, 'cbar': True, 'cbar_ax': [axes[8], axes[9]]},
        {'ax': axes[9], 'data': ds_walker.cveg_pot, 'vmax': vmax_veg, 'cbar': False},
        {'ax': axes[10], 'data': ds_walker.csoil_act, 'vmax': vmax_soil * 4, 'cbar': True, 'cbar_ax': [axes[10], axes[11]]},
        {'ax': axes[11], 'data': ds_walker.csoil_pot, 'vmax': vmax_soil * 4, 'cbar': False},
        {'ax': axes[12], 'data': ds_erb.erb_cveg_act, 'vmax': vmax_veg, 'cbar': True, 'cbar_ax': [axes[12], axes[13]]},
        {'ax': axes[13], 'data': ds_erb.erb_cveg_pot, 'vmax': vmax_veg, 'cbar': False},
        {'ax': axes[18], 'data': ds_sand.sand_soc_act, 'vmax': vmax_soil, 'cbar': True, 'cbar_ax': [axes[18], axes[19]]},
        {'ax': axes[19], 'data': ds_sand.sand_soc_pot, 'vmax': vmax_soil, 'cbar': False},
    ]

    # Dictionary with axes that will have data
    used_axes_indices = {config['ax'] for config in plot_configs}

    # Loop through plot_configs list and create subplots
    for config in plot_configs:
        cbar_axes = config.get('cbar_ax', False)
        plot_sub(fig, config['ax'], config['data'], config['vmax'], *coords, 
                 'gist_earth_r', config['cbar'], cbar_axes)

    # Subplot layout 
    for ax in axes:       
        # Layout of subplots with data
        if ax in used_axes_indices:
                ax.grid(zorder=1)
                ax.tick_params(axis='both', which='major', labelsize='medium')
        
        # Layout of subplots without data
        if ax not in used_axes_indices:
            ax.set_xlim(lon_min, lon_max)
            ax.set_ylim(lat_min, lat_max)
            ax.set_facecolor('#d9d9d9')
            ax.tick_params(axis='both', which='major', labelsize='medium', 
                           labelcolor='none', color='none')
        # General
        ax.set_xlabel('')
        ax.set_ylabel('')
        ax.yaxis.set_major_locator(MaxNLocator(integer=True))
        ax.xaxis.set_major_locator(MaxNLocator(integer=True))
        
        # Remove y-ticks and ticklabels of selected subplots
        if ax in [axes[i] for i in [1, 3, 5, 7, 9, 11, 13, 15, 17, 19]]:
            ax.tick_params(axis='y', left=False, labelleft=False)
    
    # Small text boxes inside subplots
    for i in [axes[2], axes[3], axes[18], axes[19]]:
        i.text(0.02, 0.02, '0–30 cm', transform=i.transAxes,
               ha='left', va='bottom', size='medium', zorder=3)
        i.add_patch(patches.Rectangle(
            (lon_min,lat_min), 0.9, 0.32, 
            color='#ffffff', alpha=0.75, lw=1, ec='#000000', zorder=2))
    for i in [axes[4], axes[5]]:
        i.text(0.02, 0.02, 'Forest biomass only', transform=i.transAxes,
               ha='left', va='bottom', size='medium', zorder=3)
        i.add_patch(patches.Rectangle(
            (lon_min,lat_min), 2, 0.32, 
            color='#ffffff', alpha=0.75, lw=1, ec='#000000', zorder=2))        
    for i in [axes[10], axes[11]]:
        i.text(0.02, 0.02, '0–200 cm', transform=i.transAxes,
               ha='left', va='bottom', size='medium', zorder=3)
        i.add_patch(patches.Rectangle(
            (lon_min,lat_min), 1, 0.32, 
            color='#ffffff', alpha=0.75, lw=1, ec='#000000', zorder=2))
    for i in [axes[6], axes[7]]:
        i.text(0.02, 0.02, 'From Sanderman\net al. 2017', transform=i.transAxes,
               ha='left', va='bottom', size='medium')
    for i in [axes[14], axes[15], axes[16], axes[17]]:
        i.text(0.02, 0.02, 'NA', transform=i.transAxes,
               ha='left', va='bottom', size='medium')
        
    # Column titles
    axes[0].set_title('Actual\nAGBC + BGBC', size='medium')
    axes[1].set_title('Potential\nAGBC + BGBC', size='medium')
    axes[2].set_title('Actual\nSOC', size='medium')
    axes[3].set_title('Potential\nSOC', size='medium')

    # Row titles
    if name_out is not None and name_out.startswith('fig04'):
        l_mo = 'Mo et al.\n2023$^{10}$'
        l_walker = 'Walker et al.\n2022$^{8}$' 
        l_erb = 'Erb et al.\n2018$^{6}$'
        l_sand = 'Sanderman\net al. 2017$^{7}$'
    elif name_out is not None and name_out.startswith('figs03'):
        l_mo = 'Mo et al.\n2023 [S1]'
        l_walker = 'Walker et al.\n2022 [S2]' 
        l_erb = 'Erb et al.\n2018 [S3]' 
        l_sand = 'Sanderman\net al. 2017 [S4]'
    else:
        l_mo = 'Mo et al.\n2023'
        l_walker = 'Walker et al.\n2022' 
        l_erb = 'Erb et al.\n2018' 
        l_sand = 'Sanderman\net al. 2017'
  
    axes[0].set_ylabel('This\nstudy', size='medium')
    axes[4].set_ylabel(l_mo, size='medium')
    axes[8].set_ylabel(l_walker, size='medium')
    axes[12].set_ylabel(l_erb, size='medium')
    axes[16].set_ylabel(l_sand, size='medium')

    # Export
    if name_out != None:
        plt.savefig(os.path.join(dir06, f'{name_out}.pdf'), dpi=600, bbox_inches='tight', pad_inches=0.1)
        plt.savefig(os.path.join(dir06, f'{name_out}.png'), dpi=600, bbox_inches='tight', pad_inches=0.1);

In [ ]:
# Alps
plot_example(45.9, 47.9, 8.9, 10.9, 190, 200, 'fig04_spatial_example_alps_map')

In [ ]:
# Amazon
plot_example(-0.1, 1.9, -61.1, -59.1, 275, 80, 'figs03_spatial_example_brazil_map')

In [ ]:
# Amazon
plot_example(-10.5, -8.5, -68, -66, 320, 100)

In [ ]:
# Amazon
plot_example(-10.5, -8.5, -67, -65, 320, 100)

In [ ]:
# Nile
plot_example(27, 29, 30, 32, 50, 100)

In [ ]:
# Bangladesh
plot_example(21.5, 23.5, 88, 90, 200, 200)

In [ ]:
# Bangladesh
plot_example(21, 26, 110, 115, 250, 200)

In [ ]:
# Amazon
plot_example(-8, -7, -73, -72, 320, 100)

In [ ]:
# Amazon
plot_example(-4.5, -2.5, -54, -52, 320, 100)

In [ ]:
# Los nevados
plot_example(4, 6, -76, -74, 250, 300)

In [ ]:
# Los nevados
plot_example(4, 6, -76, -74, 250, 300)

In [ ]:
# Colorado
plot_example(38.5, 40.5, -106, -104, 150, 200)

In [ ]:
# Nepal
plot_example(27, 29, 84, 86, 200, 200)

In [ ]:
# Nepal
plot_example(27, 29, 87, 89, 200, 200)

In [ ]:
# Malaysia
plot_example(0, 2, 111, 113, 300, 200)

In [ ]:
# Kenia
plot_example(-1, 1, 36, 38, 250, 200)

In [ ]:
# Ural
plot_example(65, 67, 59, 61, 100, 250)

In [ ]:
# Iceland
plot_example(64, 66, -21, -19, 50, 200)

In [ ]:
# Vancouver
plot_example(50, 52, -125, -123, 250, 200)

In [ ]:
# Titicaca
plot_example(-17, -15, -69, -67, 250, 200)

In [ ]:
cluster.close()